`` https://mne.tools/stable/auto_tutorials/sample-datasets/plot_sleep.html#sphx-glr-auto-tutorials-sample-datasets-plot-sleep-py = https://mne.tools/dev/auto_tutorials/sample-datasets/plot_sleep.html#sphx-glr-auto-tutorials-sample-datasets-plot-sleep-py ``

`https://pyedflib.readthedocs.io/en/latest/ref/edfreader.html`

In [ ]:
!pip install mne
import mne

     |████████████████████████████████| 7.0MB 4.8MB/s 


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [ ]:
%cd /content/drive/MyDrive/Colab Notebooks/TFM

/content/drive/MyDrive/Colab Notebooks/TFM


In [ ]:
%ls

 10_epochs_overview.ipynb   notes.txt           PSG2_ann.csv
 60_sleep.ipynb            'PROJECTE PSG.pdf'   PSG2.edf
'Copia di Untitled'         PSG1_ann.csv        test_generator_2.edf
 edf_file.edf               PSG1_ann.gsheet     Untitled
'lectura fases sueño.pdf'   PSG1.edf


In [ ]:
# files = ['PSG1.edf', 'PSG2.edf']
# for f in files:
  # data = mne.io.read_raw_edf(f)
  # raw_data = data.get_data()

file = "PSG1.edf"
data = mne.io.read_raw_edf(file) # 13569536
raw_data = data.get_data() # 50 x 13569536

Extracting EDF parameters from /content/drive/MyDrive/Colab Notebooks/TFM/PSG1.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


<ipython-input-5-8bed9ec4d111>:7: RuntimeWarning: Omitted 3 annotation(s) that were outside data range.
  data = mne.io.read_raw_edf(file) # 13569536
<ipython-input-5-8bed9ec4d111>:7: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  data = mne.io.read_raw_edf(file) # 13569536


In [ ]:
# file2 = "PSG2.edf"
# data2 = mne.io.read_raw_edf(file2) # 12939776
# raw_data2 = data2.get_data() # 50 x 12939776 

In [ ]:
# raw_data_tot = np.concatenate(raw_data, raw_data2)
info = data.info
channels = data.ch_names

In [ ]:
annot_train = mne.read_annotations(file) # data.annotations # PSG1_ann.csv # 3041 # annot_train[0]['onset']

In [ ]:
sleep_stages_targets = ['Sleep stage N1', 'Sleep stage N2', 'Sleep stage N3', 'Sleep stage W', 'Sleep stage R']
indices =[index for index, item in enumerate(annot_train) if item['description'] in sleep_stages_targets] # 884
annotation_train_indexed = annot_train[indices] 
labels = annotation_train_indexed.to_data_frame()['description'].to_numpy()

In [ ]:
onsets = [] 
for i in annotation_train_indexed:
  onsets.append(i['onset']) 

In [ ]:
signals_queued = [] # 44200 items
t_epoch = 30 # s
for o in map(int, onsets):
  for c in range(len(channels)):
    signals_queued.append(raw_data[c][o:o+t_epoch])

In [ ]:
signals = [] # 884
n_channels = len(channels)
for i in range(0,len(signals_queued),n_channels):
  signals.append(signals_queued[i:i+n_channels])

In [ ]:
df = pd.DataFrame(data={'signals': signals}) 
df['labels']=annotation_train_indexed.to_data_frame()['description']

In [ ]:
df

,signals,labels
0,"[[-0.010799999999999999, -0.010799999999999999...",Sleep stage W
1,"[[-0.010799999999999999, -0.010799999999999999...",Sleep stage W
2,"[[-0.010799999999999999, -0.010799999999999999...",Sleep stage W
3,"[[-0.010799999999999999, -0.010799999999999999...",Sleep stage W
4,"[[-0.010799999999999999, -0.010799999999999999...",Sleep stage W
...,...,...
879,"[[-0.00017781643396658255, -0.0001722133211261...",Sleep stage W
880,"[[-0.00017023575188830376, -0.0001603479056992...",Sleep stage W
881,"[[-0.00012771801327534886, -0.0001053055619134...",Sleep stage W
882,"[[-9.40993362325473e-05, -0.000105305561913481...",Sleep stage W


In [ ]:
channels_to_detect = ['O2','LOC']
channels_indices = []
for c in channels_to_detect:
  channels_indices.append(channels.index(c))

In [ ]:
signals_O2_and_LOC = []
 
for i in df['signals']:
  accessed_mapping = map(i.__getitem__, channels_indices)
  signals_O2_and_LOC.append(list(accessed_mapping))

In [ ]:
df_O2_and_LOC = pd.DataFrame(data={'signals': signals_O2_and_LOC})
df_O2_and_LOC['labels'] = annotation_train_indexed.to_data_frame()['description']
 
df_O2_and_LOC 

,signals,labels
0,"[[-0.0042667704280155636, -0.00426677042801556...",Sleep stage W
1,"[[-0.0042176607919432365, -0.00421930876630807...",Sleep stage W
2,"[[-0.004151741817349508, -0.004155696955825131...",Sleep stage W
3,"[[-0.004116804760814831, -0.004119111924925612...",Sleep stage W
4,"[[-0.004059784847791256, -0.004062751201647974...",Sleep stage W
...,...,...
879,"[[3.2794689860380116e-05, 4.03753719386589e-05...",Sleep stage W
880,"[[5.0592813000686825e-05, 4.696726939803175e-0...",Sleep stage W
881,"[[1.3678187228198843e-05, 6.427100022888704e-0...",Sleep stage W
882,"[[4.004577706569026e-05, 4.8615243762874965e-0...",Sleep stage W


In [ ]:
len(df_O2_and_LOC.iloc[0]['signals'][0])

30

In [ ]:
# len(segnales) 884 # len(df2['signals'][0]) 50 # len(df2['signals'][0][0]) 30 # signals_total[0:50] == signals[0]

# signals_tot = []
# for c in range(len(channels)):
#   for o in onsets:
#     signals_tot.append(raw_data[c][int(o):int(o)+30])